In [2]:
#@title Funciones de Informacion y Sugerencias
!pip install deep_translator
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import pandas as pd
from google.colab.patches import cv2_imshow
import cv2
import os
from datetime import date,datetime
import csv
from deep_translator import GoogleTranslator



CLASS_NAMES = {
        0: 'apple_pie', 1: 'baby_back_ribs', 2: 'baklava', 3: 'beef_carpaccio',
        4: 'beef_tartare', 5: 'beet_salad', 6: 'beignets', 7: 'bibimbap',
        8: 'bread_pudding', 9: 'breakfast_burrito', 10: 'bruschetta', 11: 'caesar_salad',
        12: 'cannoli', 13: 'caprese_salad', 14: 'carrot_cake', 15: 'ceviche',
        16: 'cheese_plate', 17: 'cheesecake', 18: 'chicken_curry', 19: 'chicken_quesadilla',
        20: 'chicken_wings', 21: 'chocolate_cake', 22: 'chocolate_mousse', 23: 'churros',
        24: 'clam_chowder', 25: 'club_sandwich', 26: 'crab_cakes', 27: 'creme_brulee',
        28: 'croque_madame', 29: 'cup_cakes', 30: 'deviled_eggs', 31: 'donuts',
        32: 'dumplings', 33: 'edamame', 34: 'eggs_benedict', 35: 'escargots',
        36: 'falafel', 37: 'filet_mignon', 38: 'fish_and_chips', 39: 'foie_gras',
        40: 'french_fries', 41: 'french_onion_soup', 42: 'french_toast', 43: 'fried_calamari',
        44: 'fried_rice', 45: 'frozen_yogurt', 46: 'garlic_bread', 47: 'gnocchi',
        48: 'greek_salad', 49: 'grilled_cheese_sandwich', 50: 'grilled_salmon',
        51: 'guacamole', 52: 'gyoza', 53: 'hamburger', 54: 'hot_and_sour_soup',
        55: 'hot_dog', 56: 'huevos_rancheros', 57: 'hummus', 58: 'ice_cream',
        59: 'lasagna', 60: 'lobster_bisque', 61: 'lobster_roll_sandwich',
        62: 'macaroni_and_cheese', 63: 'macarons', 64: 'miso_soup', 65: 'mussels',
        66: 'nachos', 67: 'omelette', 68: 'onion_rings', 69: 'oysters', 70: 'pad_thai',
        71: 'paella', 72: 'pancakes', 73: 'panna_cotta', 74: 'peking_duck', 75: 'pho',
        76: 'pizza', 77: 'pork_chop', 78: 'poutine', 79: 'prime_rib',
        80: 'pulled_pork_sandwich', 81: 'ramen', 82: 'ravioli', 83: 'red_velvet_cake',
        84: 'risotto', 85: 'samosa', 86: 'sashimi', 87: 'scallops', 88: 'seaweed_salad',
        89: 'shrimp_and_grits', 90: 'spaghetti_bolognese', 91: 'spaghetti_carbonara',
        92: 'spring_rolls', 93: 'steak', 94: 'strawberry_shortcake', 95: 'sushi',
        96: 'tacos', 97: 'takoyaki', 98: 'tiramisu', 99: 'tuna_tartare', 100: 'waffles'
    }


def modelo(nombre):
    IA = tf.keras.models.load_model(nombre)
    print("modelo cargado exitosamente")
    return IA

def prediccion(IA, imagen):
  img = imagen
  img_redimension = image.load_img(img, target_size=(150, 150))
  img_array = image.img_to_array(img_redimension)/255
  img_array = np.expand_dims(img_array, axis=0)

  prediccion = IA.predict(img_array)

  predict = zip(prediccion[0],list(range(0,101)))
  predict = list(predict)
  ordenados = sorted(predict, key=lambda predict : predict[0],reverse=True)
  print(ordenados)
  prediccion_numerica = np.argmax(prediccion, axis=1)[0]

  comida = CLASS_NAMES[prediccion_numerica]

  return comida


def lectura(): #tener los archivos en la misma ruta donde se ejecuta el codigo
  data_nutricion = pd.read_csv('nutrition2.csv')
  data_excel = pd.read_excel('tabla.xlsx')


  print("se leyeron ambos archivos correctamente")

  #procesado a data excel y a data nutricion

  data_nutri = data_nutricion[data_nutricion["weight"] == 100]


  filas = data_excel.iloc[1, :]
  data_excel.columns = filas
  data_excel.reset_index(inplace=True)
  data_excel.drop([0,1,2], inplace=True)
  data_excel.reset_index(inplace=True)
  data_excel.rename(columns={"Proteína (g)":"Proteina", "Calorías": "Calorias"}, inplace=True)


  data_excel.drop(["index", "level_0", "Codigo", "Humed. (g)", "Fósforo (mg)", "Potasio (mg)", "Grasas (g)", np.nan], inplace=True, axis=1)

  print("procesado del excel y la tabla nutricion exitosamente")

  return data_nutri, data_excel

def busqueda(predicion, data_nutri):
  informacion = data_nutri[data_nutri["label"] == predicion]
  informacion.reset_index(inplace=True)

  return informacion

def guardado(fecha_actual,informacion):

    #directorio_base=os.getcwd()+'\\registro_de_alimentacion.csv' USAR SOLO SI NO SE ESTA EJECUTANDO EN LA MISMA RUTA
    nombre_registro = "registro_de_alimentacion.csv"
    formato=datetime.strftime(fecha_actual,'%d/%m/%Y')

    informacion.insert(0,'Fechas',[formato])

    if not os.path.exists(nombre_registro):

        with open('registro_de_alimentacion.csv', mode='a' ,newline='') as registro:
            archivo=csv.writer(registro)
            archivo.writerow(list(informacion))
            archivo.writerow(list(informacion.loc[0,list(informacion.columns)]))
    else:

        with open('registro_de_alimentacion.csv', mode='a' ,newline='') as registro:
            archivo=csv.writer(registro)
            archivo.writerow(list(informacion.loc[0,list(informacion.columns)]))

def traduccion(prediccion):
  traduccion = GoogleTranslator(source='auto', target='es').translate(prediccion.replace("_", " "))
  return traduccion

def sugerencias1(informacion, data_excel):
    nombre = informacion["label"][0]
    nombre_traducido = traduccion(nombre)
    df = data_excel

    if informacion["sugars"][0] > 15:

        print(f"{nombre_traducido} tiene alto porcentaje de azucar, evitar su consumo excesivo")
    else:

        if informacion["calories"][0] <= 400 and informacion["protein"][0] >= 20:

            if informacion["carbohydrates"][0] > 45:
              print(f"{nombre_traducido} tiene buen porcentaje en proteinas y carbohidratos pero es bajo en calorias, aqui unas sugerencias con que complementarlo: ")
              recomendaciones = df[(df['Calorias'] > 300) & (df['Proteina'] < 20)]

            else:
              print(f"{nombre_traducido} tiene buen porcentaje en proteinas pero es bajo en calorias y carbohidratos, aqui unas sugerencias con que complementarlo: ")
              recomendaciones = df[(df['Calorias'] > 300) & (df['Carbohidratos'] > 25)]

        elif informacion["calories"][0] > 100 and informacion["protein"][0] < 20:

            if informacion["carbohydrates"][0] > 45:
                print(f"{nombre_traducido} tiene un buen porcentaje de calorias y carbohidratos pero es bajo en proteinas, aqui unas sugerencias con que complementarlo: ")
                recomendaciones = df[(df['Proteina'] > 20) & (df['Calorias'] < 100)]
            else:
                print(f"{nombre_traducido} tiene un buen porcentaje de calorias pero es bajo en proteinas y carbohidratos, aqui unas sugerencias con que complementarlo: ")
                recomendaciones = df[(df['Proteina'] > 20) & (df['Carbohidratos'] > 25)]

        elif informacion["calories"][0] < 300 and informacion["protein"][0] < 20:

            if informacion["carbohydrates"][0] > 45:
                print(f"{nombre_traducido} tiene buen porcentaje de carbohidratos pero es bajo en calorias y en proteinas, te recomendamos aumentar ambas con unos alimentos sugeridos: ")
                recomendaciones = df[(df['Calorias'] > 300) & (df['Proteina'] > 20)]
            else:
                print(f"{nombre_traducido} es bajo en calorias, en proteinas y en carbohidratos, te recomendamos aumentar ambas con unos alimentos sugeridos: ")
                recomendaciones = df[(df['Calorias'] > 300) & (df['Proteina'] > 20) & (df['Carbohidratos'] > 25)]
        else:

            print(f"{nombre_traducido} es alto en calorias y en proteinas pero bajo en carbohidratos, aqui unas sugerencias para complementarlo: ")
            recomendaciones = df[(df['Carbohidratos'] > 25) & (df['Calorias'] < 300) & (df['Proteina'] < 20)]


        if not recomendaciones.empty:
          recomendaciones = recomendaciones.sample(n=min(3, len(recomendaciones)))
          #print(recomendaciones)
          print("Te recomendamos las siguientes comidas:")
          for _, row in recomendaciones.iterrows():
              nombre_recomendado = row['Alimento']
              print(f"- {nombre_recomendado} (Calorías: {row['Calorias']}, Proteínas: {row['Proteina']}, Carbohidratos: {row['Carbohidratos']})")

def sugerencias(informacion, data_excel):
    # Diccionario para almacenar la salida
    resultado = {
        "descripcion": "",
        "sugerencias": ""
    }

    nombre = informacion["label"][0]
    nombre_traducido = traduccion(nombre)
    df = data_excel

    if informacion["sugars"][0] > 15:
        resultado["descripcion"] = f"{nombre_traducido} tiene alto porcentaje de azúcar, evitar su consumo excesivo."
    else:
        if informacion["calories"][0] <= 400 and informacion["protein"][0] >= 20:
            if informacion["carbohydrates"][0] > 45:
                resultado["descripcion"] = f"{nombre_traducido} tiene buen porcentaje en proteínas y carbohidratos pero es bajo en calorías. Aquí unas sugerencias con qué complementarlo:"
                recomendaciones = df[(df['Calorias'] > 300) & (df['Proteina'] < 20)]
            else:
                resultado["descripcion"] = f"{nombre_traducido} tiene buen porcentaje en proteínas pero es bajo en calorías y carbohidratos. Aquí unas sugerencias con qué complementarlo:"
                recomendaciones = df[(df['Calorias'] > 300) & (df['Carbohidratos'] > 25)]

        elif informacion["calories"][0] > 100 and informacion["protein"][0] < 20:
            if informacion["carbohydrates"][0] > 45:
                resultado["descripcion"] = f"{nombre_traducido} tiene un buen porcentaje de calorías y carbohidratos pero es bajo en proteínas. Aquí unas sugerencias con qué complementarlo:"
                recomendaciones = df[(df['Proteina'] > 20) & (df['Calorias'] < 100)]
            else:
                resultado["descripcion"] = f"{nombre_traducido} tiene un buen porcentaje de calorías pero es bajo en proteínas y carbohidratos. Aquí unas sugerencias con qué complementarlo:"
                recomendaciones = df[(df['Proteina'] > 20) & (df['Carbohidratos'] > 25)]

        elif informacion["calories"][0] < 300 and informacion["protein"][0] < 20:
            if informacion["carbohydrates"][0] > 45:
                resultado["descripcion"] = f"{nombre_traducido} tiene buen porcentaje de carbohidratos pero es bajo en calorías y en proteínas. Te recomendamos aumentar ambas con unos alimentos sugeridos:"
                recomendaciones = df[(df['Calorias'] > 300) & (df['Proteina'] > 20)]
            else:
                resultado["descripcion"] = f"{nombre_traducido} es bajo en calorías, en proteínas y en carbohidratos. Te recomendamos aumentar ambas con unos alimentos sugeridos:"
                recomendaciones = df[(df['Calorias'] > 300) & (df['Proteina'] > 20) & (df['Carbohidratos'] > 25)]
        else:
            resultado["descripcion"] = f"{nombre_traducido} es alto en calorías y en proteínas pero bajo en carbohidratos. Aquí unas sugerencias para complementarlo:"
            recomendaciones = df[(df['Carbohidratos'] > 25) & (df['Calorias'] < 300) & (df['Proteina'] < 20)]

        # Construir la lista de sugerencias
        if not recomendaciones.empty:
            recomendaciones = recomendaciones.sample(n=min(3, len(recomendaciones)))
            sugerencias_texto = "Te recomendamos las siguientes comidas:\n"
            for _, row in recomendaciones.iterrows():
                nombre_recomendado = row['Alimento']
                sugerencias_texto += f"- {nombre_recomendado} (Calorías: {row['Calorias']}, Proteínas: {row['Proteina']}, Carbohidratos: {row['Carbohidratos']})\n"
            resultado["sugerencias"] = sugerencias_texto

    return resultado

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.5 MB/s eta 0:00:00


In [3]:
#@title APP FINAL
!pip install -q gradio tensorflow pillow pandas matplotlib

import gradio as gr
import tensorflow as tf
from PIL import Image
import numpy as np
import os
import pandas as pd
from datetime import datetime, date
import csv
import matplotlib.pyplot as plt
import matplotlib.dates as mdates


MODEL_PATH = 'apa.h5'
NUTRITION_CSV = 'nutrition2.csv'
REGISTRO_CSV = 'registro.csv'
data_nutri, data_excel = lectura() #dataframes

# Verificar archivos esenciales primero
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(" Archivo del modelo 'apa.h5' no encontrado")
if not os.path.exists(NUTRITION_CSV):
    raise FileNotFoundError(" Archivo nutricional 'nutrition2.csv' no encontrado")

# Cargar datos nutricionales una sola vez
try:
    df_nutricion = pd.read_csv(NUTRITION_CSV)
    df_nutricion = df_nutricion[df_nutricion['weight'] == 100]
except Exception as e:
    raise ValueError(f"Error cargando datos nutricionales: {str(e)}")


model = tf.keras.models.load_model(MODEL_PATH)
print(" Modelo cargado exitosamente")

# Mapeo completo de clases (asegúrate de completar todas las 101 clases)
CLASS_NAMES = {
        0: 'apple_pie', 1: 'baby_back_ribs', 2: 'baklava', 3: 'beef_carpaccio',
        4: 'beef_tartare', 5: 'beet_salad', 6: 'beignets', 7: 'bibimbap',
        8: 'bread_pudding', 9: 'breakfast_burrito', 10: 'bruschetta', 11: 'caesar_salad',
        12: 'cannoli', 13: 'caprese_salad', 14: 'carrot_cake', 15: 'ceviche',
        16: 'cheese_plate', 17: 'cheesecake', 18: 'chicken_curry', 19: 'chicken_quesadilla',
        20: 'chicken_wings', 21: 'chocolate_cake', 22: 'chocolate_mousse', 23: 'churros',
        24: 'clam_chowder', 25: 'club_sandwich', 26: 'crab_cakes', 27: 'creme_brulee',
        28: 'croque_madame', 29: 'cup_cakes', 30: 'deviled_eggs', 31: 'donuts',
        32: 'dumplings', 33: 'edamame', 34: 'eggs_benedict', 35: 'escargots',
        36: 'falafel', 37: 'filet_mignon', 38: 'fish_and_chips', 39: 'foie_gras',
        40: 'french_fries', 41: 'french_onion_soup', 42: 'french_toast', 43: 'fried_calamari',
        44: 'fried_rice', 45: 'frozen_yogurt', 46: 'garlic_bread', 47: 'gnocchi',
        48: 'greek_salad', 49: 'grilled_cheese_sandwich', 50: 'grilled_salmon',
        51: 'guacamole', 52: 'gyoza', 53: 'hamburger', 54: 'hot_and_sour_soup',
        55: 'hot_dog', 56: 'huevos_rancheros', 57: 'hummus', 58: 'ice_cream',
        59: 'lasagna', 60: 'lobster_bisque', 61: 'lobster_roll_sandwich',
        62: 'macaroni_and_cheese', 63: 'macarons', 64: 'miso_soup', 65: 'mussels',
        66: 'nachos', 67: 'omelette', 68: 'onion_rings', 69: 'oysters', 70: 'pad_thai',
        71: 'paella', 72: 'pancakes', 73: 'panna_cotta', 74: 'peking_duck', 75: 'pho',
        76: 'pizza', 77: 'pork_chop', 78: 'poutine', 79: 'prime_rib',
        80: 'pulled_pork_sandwich', 81: 'ramen', 82: 'ravioli', 83: 'red_velvet_cake',
        84: 'risotto', 85: 'samosa', 86: 'sashimi', 87: 'scallops', 88: 'seaweed_salad',
        89: 'shrimp_and_grits', 90: 'spaghetti_bolognese', 91: 'spaghetti_carbonara',
        92: 'spring_rolls', 93: 'steak', 94: 'strawberry_shortcake', 95: 'sushi',
        96: 'tacos', 97: 'takoyaki', 98: 'tiramisu', 99: 'tuna_tartare', 100: 'waffles'
    }


def buscar_comida(comida: str) -> pd.DataFrame:
    """Busca información nutricional para una comida específica"""
    resultado = df_nutricion[df_nutricion['label'] == comida]
    print(resultado)
    if resultado.empty:
        raise ValueError(f"Comida '{comida}' no encontrada en la base de datos")
    return resultado

def actualizar_registro(fecha: date, datos: pd.DataFrame):
    """Actualiza el registro histórico de nutrición"""
    datos = datos.copy()
    datos.insert(0, 'Fecha', fecha.strftime('%d/%m/%Y'))

    # Crear archivo si no existe
    if not os.path.exists(REGISTRO_CSV):
        datos.to_csv(REGISTRO_CSV, index=False)
    else:
        datos.to_csv(REGISTRO_CSV, mode='a', header=False, index=False)

def generar_grafico():
    """Genera el gráfico nutricional desde el registro"""
    if not os.path.exists('registro_de_alimentacion.csv'):
        return None

    try:
        df = pd.read_csv('registro.csv', parse_dates=['Fecha'])
        df = df.sort_values('Fecha')

        plt.figure(figsize=(10, 6))
        plt.plot(df['Fecha'], df['calories'], label='Calorías', color='blue')
        plt.plot(df['Fecha'], df['protein'], label='Proteínas', color='green')
        plt.plot(df['Fecha'], df['carbohydrate'], label='Carbohidratos', color='red')
        plt.plot(df['Fecha'], df['sugars'], label='Azúcares', color='purple')

        plt.title('Historial Nutricional')
        plt.xlabel('Fecha')
        plt.ylabel('Cantidad (por 100g)')
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d/%m/%Y'))
        plt.xticks(rotation=45)
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        return plt.gcf()
    except Exception as e:
        print(f"Error generando gráfico: {str(e)}")
        return None


def predict_image(img):
    try:
        # Preprocesamiento de imagen
        img = img.resize((150, 150))
        img_array = np.array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        # Predicción del modelo
        preds = model.predict(img_array, verbose=0)[0]
        top_indices = np.argsort(preds)[::-1][:5]
        top_predicciones = {CLASS_NAMES[i]: float(preds[i]) for i in top_indices}
        comida_principal = CLASS_NAMES[top_indices[0]]

        # Obtener información nutricional
        info_comida = buscar_comida(comida_principal)

        # Actualizar registro y generar gráfico
        actualizar_registro(date.today(), info_comida)
        grafico = generar_grafico()

        # Preparar datos para salida
        info_nutricional = info_comida[['calories', 'protein', 'carbohydrates', 'sugars']]
        informacion = busqueda(CLASS_NAMES[top_indices[0]], data_nutri) #data frame alimento

        k=dict(sugerencias(informacion, data_excel)) #recomendaciones
        print(top_predicciones)
        print(k)
        u=k['descripcion']
        v=k['sugerencias']
        return top_predicciones, info_nutricional,u,v

    except Exception as e:
        error_msg = f"⚠ Error: {str(e)}"
        return error_msg, pd.DataFrame(), None


interfaz = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(label="🍴 Sube tu imagen de comida", type="pil"),
    outputs=[
        gr.Label(label="🔍 Predicciones Top 5"),
        gr.DataFrame(label="📊 Información Nutricional (por 100g)"),
        #gr.Plot(label="📈 Evolución Nutricional"),
        gr.Textbox(label="🍴 Descripcion del Alimento 🍴",interactive=False),
        gr.Textbox(label="🍎 Sugerencias 🍎",interactive=False),
    ],
    title="🍎 Analizador de Alimentos Inteligente",
    description="Clasificación de alimentos con análisis nutricional e histórico gráfico",
    examples=[
        ["ejemplo1.jpg"],
        ["ejemplo2.jpg"]
    ] if os.path.exists("ejemplo1.jpg") else None,
    theme=gr.themes.Soft(),
    allow_flagging="never"
)

interfaz.launch(share=True, debug=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 107.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 whi

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


 Modelo cargado exitosamente


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7f4f4bf53309ca7d7c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


             label  weight  calories  protein  carbohydrates  fats  fiber  \
225  frozen_yogurt     100       150        4             25     3      0   

     sugars  sodium  
225      20      50  
{'frozen_yogurt': 0.9631388187408447, 'ice_cream': 0.012600007466971874, 'churros': 0.01095480378717184, 'sashimi': 0.0023473792243748903, 'ceviche': 0.0021317352075129747}
{'descripcion': 'yogurt congelado tiene alto porcentaje de azúcar, evitar su consumo excesivo.', 'sugerencias': ''}
             label  weight  calories  protein  carbohydrates  fats  fiber  \
225  frozen_yogurt     100       150        4             25     3      0   

     sugars  sodium  
225      20      50  
{'frozen_yogurt': 0.9631388187408447, 'ice_cream': 0.012600007466971874, 'churros': 0.01095480378717184, 'sashimi': 0.0023473792243748903, 'ceviche': 0.0021317352075129747}
{'descripcion': 'yogurt congelado tiene alto porcentaje de azúcar, evitar su consumo excesivo.', 'sugerencias': ''}
                 label  